TextCNN利用CNN（卷积神经网络）进行**文本特征抽取**，不同大小的卷积核分别抽取n-gram特征，卷积计算出的**特征图**经过MaxPooling保留最大的特征值，然后**拼接成一个向量作为文本的表示**。

这里我们基于TextCNN原始论文的设定，分别采用了100个大小为2,3,4的卷积核，最后得到的文本向量大小为100*3=300维

### 一、十折验证数据准备

In [1]:
import logging
import random
import numpy as np
import torch
import pandas as pd

logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')

# set seed
seed = 666
random.seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

# set cuda
gpu = 0
# 右边是个bool判断，返回True或者False
use_cuda = gpu >= 0 and torch.cuda.is_available()
if use_cuda:
    torch.cuda.set_device(gpu)
    device = torch.device("cuda", gpu)
else:
    device = torch.device("cpu")
logging.info("Use cuda: %s, gpu id: %d.", use_cuda, gpu)

2020-07-31 11:46:49,573 INFO: Use cuda: True, gpu id: 0.


#### 统一的十折交叉验证（训练集）数据的加载方式

In [2]:
n_fold = 10
n_example = 2000
data_file = './train_set.csv.zip'
train = pd.read_csv(data_file, sep='\t', nrows=n_example)

In [3]:
# 数据直接全局变量
train_texts = train['text'].tolist()
train_labels = train['label'].tolist()
n_total = len(train_labels)

In [4]:
def all_data2index(fold_num):
    """读取df，将series转换为list进行处理"""
        
    # 1.所有数据打乱，通过打乱列表的索引来实现 / 通过sklearn的shuffle模块？
    index = list(range(n_total))
    np.random.shuffle(index)
    all_texts = []
    all_labels = []
    for i in index:
        all_texts.append(train_texts[i])
        all_labels.append(train_labels[i])
    
    # 2.将所有数据按照类别进行划分，通过索引实现：字典检查某个键是否存在，不存在，就创建列表，存在则往列表里添加
    label2id = {}
    for i in range(n_total):
        label = str(all_labels[i])
        # 字典检查某个键是否存在？？ 不加.keys?
        if label not in label2id:
            label2id[label] = [i]
        else:
            label2id[label].append(i)
    
    # 3.
    all_index = [[] for _ in range(fold_num)]
    for label, data in label2id.items():
        # print(label, len(data))
        batch_size = int(len(data) / fold_num)
        other = len(data) - batch_size * fold_num
        
        # 对每个类别都进行10折划分
        for i in range(fold_num):
            # if判断用于赋值
            
            # ？
            cur_batch_size = batch_size + 1 if i < other else batch_size
            # print(cur_batch_size)
            batch_data = [data[i * batch_size + b] for b in range(cur_batch_size)]
            
            # 总共包含10个列表，每个列表都包含所有类别的数据
            all_index[i].extend(batch_data)
            # 等价于 all_index = [], all_index.append(batch_data)
            
            
    return all_texts, all_labels, all_index

all_texts, all_labels, all_index = all_data2index(n_fold)

In [5]:
def index2fold_data(all_texts, all_labels, all_index, fold_num):
    """这里的 texts、labels 是 fold_texts、fold_labels"""
    
    all_fold_data = []   
    
    # 4.根据每折的索引 划分出每折的数据，然后打乱
    # 平均每折的数据量    
    batch_size = int(n_total / fold_num)
    other_texts = []
    other_labels = []
    other_num = 0
    start = 0
    for fold in range(fold_num):
        # 每折的数据量
        num = len(all_index[fold])
        # 从所有数据索引中 索引出 每折数据 对应的text和label的索引
        texts = [all_texts[i] for i in all_index[fold]]
        labels = [all_labels[i] for i in all_index[fold]]
        
        # 如果每折的数据量 > 平均每折的数据量，对该折的数据进行缩减，只取到平均每折的数据量
        if num > batch_size:
            fold_texts = texts[:batch_size]
            fold_labels = labels[:batch_size]           
            other_texts.extend(texts[batch_size:])
            other_labels.extend(labels[batch_size:])
            other_num += num - batch_size
            
        # 如果每折的数据量 < 平均每折的数据量，则将上折剩余的数据补充到该折数据（列表的加法），直到取到平均每折的数据量
        elif num < batch_size:
            end = start + batch_size - num
            # 如果上折剩余的数据量不足以补充该折数据呢，索引就会报错啊？？？？？？？？？？？？？？？
            fold_texts = texts + other_texts[start: end]
            fold_labels = labels + other_labels[start: end]
            # 前面被补充过的数据不再使用
            start = end
        
        # 如果每折的数据量 = 平均每折的数据量，该折的数据进行缩减，只取到平均
        else:
            fold_texts = texts
            fold_labels = labels
        
        # 确保每折的数据量都等同于 平均每折的数据量
        assert batch_size == len(fold_labels)
    
    # 那多出来的数据呢？？？？？？？？？？？？？？？？？？？？？
    
        # 对该折的数据进行打乱，通过列表的索引
        fold_index = list(range(batch_size))
        np.random.shuffle(fold_index)
        shuffle_fold_texts = []
        shuffle_fold_labels = []
        for i in fold_index:
            shuffle_fold_texts.append(fold_texts[i])
            shuffle_fold_labels.append(fold_labels[i])
        
        # 将每折数据添加到 总划分数据里
        data = {'label': shuffle_fold_labels, 'text': shuffle_fold_texts}
        all_fold_data.append(data)
    
    # 记录输出 十折划分后 每折的数据量？？？
    logging.info("Fold lens %s", str([len(fold_data['label']) for fold_data in all_fold_data]))

    return all_fold_data

all_fold_datas = index2fold_data(all_texts, all_labels, all_index, n_fold)

2020-07-31 11:47:08,519 INFO: Fold lens [200, 200, 200, 200, 200, 200, 200, 200, 200, 200]


#### build train, dev, test data

In [6]:
fold_id = 9

# dev
dev_data = all_fold_datas[fold_id]

# train
train_texts = []
train_labels = []
for i in range(0, fold_id):
    data = all_fold_datas[i]
    train_texts.extend(data['text'])
    train_labels.extend(data['label'])
train_data = {'label': train_labels, 'text': train_texts}

# test
test_data_file = './test_a.csv.zip'
f = pd.read_csv(test_data_file, sep='\t')
texts = f['text'].tolist()
# 列表的扩充，全部标记为 0
test_data = {'label': [0] * len(texts), 'text': texts}

In [7]:
len(train_data)

2

In [9]:
len(train_data['label'])

1800

In [12]:
# 1800个字符串（文本）
train_data['text'][:3]

['6167 4480 736 3374 6831 6015 7255 7010 1582 2899 4893 6654 1914 6675 408 7495 1348 4559 7495 2410 2313 4464 2799 4853 1407 5491 7058 6045 2465 1110 4203 4203 2662 6167 4480 736 3374 6831 6015 7255 7010 6588 2376 1582 2899 4893 5560 6167 736 5491 1362 2662 2106 2115 3215 1939 5251 6654 1914 4464 4149 4853 3220 812 4114 6760 6983 2121 7399 23 4109 5041 2099 6675 408 3750 3771 1519 3700 1324 5814 900 6167 4480 1215 3912 1907 1939 4109 4933 6167 4480 5589 4411 5041 671 3634 6654 1914 1395 7015 900 2400 4411 3209 23 4109 5041 2099 1215 1582 5537 7419 1215 1582 5537 1110 4321 2021 4464 2799 4853 4233 23 5598 4515 4215 3317 7010 900 6654 1914 816 812 3272 1920 3750 6167 4480 1215 3641 316 4413 3226 5251 58 5702 137 2923 7160 7317 23 4109 5041 2099 6393 7212 900 6654 1914 5620 619 4464 3700 4464 5602 1519 3750 299 408 6515 6065 3370 1519 290 6235 6248 212 6571 4969 5659 264 4411 736 3374 5480 4630 133 2828 2471 900 4464 3700 5602 3370 1519 1465 5536 4936 6167 4480 4233 23 3481 550 7255 7010 

In [13]:
for text, label in zip(train_data['text'], train_data['label']):
    print(text)
    print(label)
    break

6167 4480 736 3374 6831 6015 7255 7010 1582 2899 4893 6654 1914 6675 408 7495 1348 4559 7495 2410 2313 4464 2799 4853 1407 5491 7058 6045 2465 1110 4203 4203 2662 6167 4480 736 3374 6831 6015 7255 7010 6588 2376 1582 2899 4893 5560 6167 736 5491 1362 2662 2106 2115 3215 1939 5251 6654 1914 4464 4149 4853 3220 812 4114 6760 6983 2121 7399 23 4109 5041 2099 6675 408 3750 3771 1519 3700 1324 5814 900 6167 4480 1215 3912 1907 1939 4109 4933 6167 4480 5589 4411 5041 671 3634 6654 1914 1395 7015 900 2400 4411 3209 23 4109 5041 2099 1215 1582 5537 7419 1215 1582 5537 1110 4321 2021 4464 2799 4853 4233 23 5598 4515 4215 3317 7010 900 6654 1914 816 812 3272 1920 3750 6167 4480 1215 3641 316 4413 3226 5251 58 5702 137 2923 7160 7317 23 4109 5041 2099 6393 7212 900 6654 1914 5620 619 4464 3700 4464 5602 1519 3750 299 408 6515 6065 3370 1519 290 6235 6248 212 6571 4969 5659 264 4411 736 3374 5480 4630 133 2828 2471 900 4464 3700 5602 3370 1519 1465 5536 4936 6167 4480 4233 23 3481 550 7255 7010 90

In [8]:
[0] * len([1, 2])

[0, 0]

### 二、build vocab

In [15]:
from collections import Counter
# 新的模块 transformers
from transformers import BasicTokenizer
basic_tokenizer = BasicTokenizer()

# 使用以前的tensorflow（2.0.0 alpha）和keras（2.3.1）版本时，会报错：AttributeError: module 'tensorflow.python.keras.api._v2.keras.layers' has no attribute 'LayerNormalization'
# 与transformers的版本无关，通过pip uninstall 原来的tf，再install 2.1.0版本的tf，就可以了

# 类似的问题：https://blog.csdn.net/qq_43486915/article/details/101475856
# 版本对应查看：https://docs.floydhub.com/guides/environments/

AttributeError: module 'tensorflow.python.keras.api._v2.keras.layers' has no attribute 'LayerNormalization'

In [17]:
reverse = lambda x: dict(zip(x, range(len(x))))
_id2word = ['[PAD]', '[UNK]', 'aa', 'scs']
reverse(_id2word)

{'[PAD]': 0, '[UNK]': 1, 'aa': 2, 'scs': 3}

In [27]:
from collections import Counter
# 新的模块
from transformers import BasicTokenizer
basic_tokenizer = BasicTokenizer()

class Vocab():
    
    # 测试初始化类时是否运行    
    print('测试，类 Vocab 的空白处再定义的时候就运行了')
    
    def __init__(self, train_data):
        
        # 测试初始化类时是否运行    
        print('测试，类 Vocab，函数 __init__ 运行了')
        
        # 在所有预料中，最少出现5次才计数
        self.min_count = 5
        # ？？？？？？？？？？？？
        self.pad = 0
        self.unk = 1
        # 所以word_size都是 4335+2
        self._id2word = ['[PAD]', '[UNK]']
        self._id2extword = ['[PAD]', '[UNK]']

        self._id2label = []
        self.target_names = []
        
        # 处理传入的参数，可以在这调用后面的函数！！！！！！！！！！ 
        self.build_vocab(train_data)
        
        # zip 对列表 x 里所有的单个字符迭代，目的是对每个字符进行编码 0 1 2 ...
        reverse = lambda x: dict(zip(x, range(len(x))))
        # 是个字典，键是 值是
        self._word2id = reverse(self._id2word)
        self._label2id = reverse(self._id2label)
        
        
        # 可以调用后面的函数 word_size、label_size，因为没有参数，所以不加括号吗？？？
        logging.info("Build vocab: words %d, labels %d." % (self.word_size, self.label_size))

    def build_vocab(self, data):
                    
        # 测试初始化类时是否运行    
        print('测试，类 Vocab，函数 build_vocab 运行了')
        
        # 这个新的属性可以在其他地方调用吗？？？？？？？？？？？？？？？？？？？？？
        self.word_counter = Counter()

        for text in data['text']:
            words = text.split()
            # 不需要if判断？？？？？？？？？？？？？？？？？？？
            for word in words:
                self.word_counter[word] += 1
        
        # 是遍历全部吗？？？？？？？？？？？？？？？？？？？？？？？？？？？
        for word, count in self.word_counter.most_common():
            # 出现次数不到5的word，不添加
            if count >= self.min_count:
                self._id2word.append(word)
        
        # 灵活变化！！！！！！！！！！！！！！
        label2name = {0: '科技', 1: '股票', 2: '体育', 3: '娱乐', 4: '时政', 5: '社会', 6: '教育', 7: '财经',
                      8: '家居', 9: '游戏', 10: '房产', 11: '时尚', 12: '彩票', 13: '星座'}
        
        # 不同于word？？？？？？？？？？？？？？？？？？？？？？？？？？？
        self.label_counter = Counter(data['label'])
        
        # 针对每个类别，根据字典的键（类别，int）取值
        for label in range(len(self.label_counter)):
            # count的作用，和前面的循环不接啊？？？？？？？？？？？？？？？？？？？？？
            count = self.label_counter[label]
            self._id2label.append(label)
            self.target_names.append(label2name[label])


    def load_pretrained_embs(self, embfile):
                    
        # 测试初始化类时是否运行    
        print('测试，类 Vocab，函数 load_pretrained_embs 运行了')
        
        # embfile 是 word2vec.txt文件，第一行是文件的 shape（n_examples, n_features) 
        with open(embfile, encoding='utf-8') as f:
            lines = f.readlines()
            # 默认的分隔符是空格？？？？？？？？？
            items = lines[0].split()
            # n_examples, n_features = 4337, 100
            word_count, embedding_dim = int(items[0]), int(items[1])
        
        # 嵌入层添加数据，文件的第一行是shape！！！！！
        # self._id2extword 的含义是所有的字符（后面的代码还会往里添加） 嵌入层加上它的作用？？？？？？？？？？？
        index = len(self._id2extword)
        embeddings = np.zeros((word_count + index, embedding_dim))
        for line in lines[1:]:
            values = line.split()
            # 第一个数值是哪个字符（字符代码）
            self._id2extword.append(values[0])
            vector = np.array(values[1:], dtype='float64')
            
            # 为什么都要加到嵌入层的第2行上，第一行都是0 ？？？？？？？？？？？？？？？？？？？
            embeddings[self.unk] += vector
            # 刚开始 index=2，所以从第三行开始填充（embeddings层中 index的部分不填充）
            # 不断更改嵌入层里 某层的数值，index不断增加，填充 embeddings层中 word_count的部分
            embeddings[index] = vector
            index += 1
            
        # 第二行加了 word_count 次，把 word2vec文件里的所有向量都加起来了，最后平均下
        embeddings[self.unk] = embeddings[self.unk] / word_count
        # np.std不说明axis的话，是求所有元素的std，但是嵌入层对应 extword 的部分还有很多行（除了第二行）的数据都是0啊？？？？？？？？
        embeddings = embeddings / np.std(embeddings)
        
        # ？？？？？？？？？？？？？？？？？？？？？？？？？
        reverse = lambda x: dict(zip(x, range(len(x))))
        # 这里新创建的属性，被用在了后面的方法里面！！！！！！！！！！！！！！！！！！
        self._extword2id = reverse(self._id2extword)
        
        #print(set(self._id2extword), self._id2extword)   
        print(len(set(self._id2extword)), len(self._id2extword))    
        
        # 生成字典时，相同的键值会合并！！！！！！！！！！！！！！！！
        assert len(set(self._id2extword)) == len(self._id2extword)

        return embeddings
    
    # 对每个字符进行编码！！！！！！！！
    
    # 三个转化函数，除了被转化对象 self._word2id  self._extword2id  self._label2id 不一样外，其他都一致
    def word2id(self, xs):
        """xs是？？？？？？？？？？？？？？？？？？？？"""
        
        # 测试初始化类时是否运行    
        print('测试，类 Vocab，函数 word2id 运行了')
        
        # 判断类型是否为 list，返回 Bool (参考：https://www.runoob.com/python/python-func-isinstance.html)
        if isinstance(xs, list):
            # 字典的 get 方法, x （键）存在则返回值，不存在则返回 1（参考：https://www.runoob.com/python/att-dictionary-get.html）
            return [self._word2id.get(x, self.unk) for x in xs]
        # 不是 list，单个元素？？？？？？？？？？？？？？？？
        return self._word2id.get(xs, self.unk)

    def extword2id(self, xs):
        
        # 测试初始化类时是否运行    
        print('测试，类 Vocab，函数 extword2id 运行了')
        
        if isinstance(xs, list):
            return [self._extword2id.get(x, self.unk) for x in xs]
        return self._extword2id.get(xs, self.unk)

    def label2id(self, xs):
        
        # 测试初始化类时是否运行    
        print('测试，类 Vocab，函数 label2id 运行了')
        
        if isinstance(xs, list):
            return [self._label2id.get(x, self.unk) for x in xs]
        return self._label2id.get(xs, self.unk)

    @property
    def word_size(self):
        
        # 测试初始化类时是否运行    
        print('测试，类 Vocab，函数 word_size 运行了')
        
        return len(self._id2word)
    
    # logging没用到！！！！！！！！！！
    @property
    def extword_size(self):
        
        # 测试初始化类时是否运行    
        print('测试，类 Vocab，函数 extword_size 运行了')
        
        return len(self._id2extword)

    @property
    def label_size(self):
        
        # 测试初始化类时是否运行    
        print('测试，类 Vocab，函数 label_size 运行了')
        
        return len(self._id2label)

测试，类 Vocab 的空白处再定义的时候就运行了


实例初始化时传入了参数，必会运行 __init__() 函数，不会运行开头的空白处，后面的方法中，__init__中没有提及的，都不会运行！！！

当读入的数据是前一万条时，显示的是4337，label=14，但是后面可以加载5978的词向量，**这里的4337和后面的5978之间的关系？？**

In [35]:
%%time
vocab = Vocab(train_data)

测试，类 Vocab，函数 __init__ 运行了
测试，类 Vocab，函数 build_vocab 运行了


2020-07-29 10:48:37,650 INFO: Build vocab: words 5996, labels 14.


测试，类 Vocab，函数 word_size 运行了
测试，类 Vocab，函数 label_size 运行了
Wall time: 59.5 s


In [12]:
# vocab不是类返回的变量，就是类本身，和以前的模型初始化一样，可以使用类中的一些方法处理变量（clf= model(), clf.fit()）！！！！！！！
# 类初始化时，不会再返回传入的参数 train_data
vocab

### 三、模块

In [4]:
import numpy as np
import torch

In [17]:
a = torch.Tensor(np.random.randn(3,2,2))
b = torch.Tensor(np.random.randn(2))

In [18]:
a.shape, b.shape

(torch.Size([3, 2, 2]), torch.Size([2]))

相乘时，首尾对应，并且首尾对应的维度都会消失（3+1-2 --> 2）！！！！！！！！！！！

乘以一维张量的，会消除一个维度

In [19]:
# TypeError: matmul(): argument 'input' (position 1) must be Tensor, not numpy.ndarray
torch.matmul(a, b).shape

torch.Size([3, 2])

首尾对应的维度都会消失（3+2-2 --> 3）

In [21]:
c = torch.Tensor(np.random.randn(3,2,2))
d = torch.Tensor(np.random.randn(2,5))
torch.matmul(c, d).shape

torch.Size([3, 2, 5])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F



In [3]:
float(-1e32)< 0

True

#### build word encoder

TextCNN的论文里，也没提到Attention word encoder啊

#### 双通道（生成的word2vec + 训练的embed-vec）

通道二

**为什么第一层全是0，第二层又有什么特殊的含义？？？？？？**

In [58]:
word2vec_path = './emb/word2vec.txt'
# 加载训练好的向量 (4335, 100)!!!!!!!!!!
extword_embed = vocab.load_pretrained_embs(word2vec_path)
# shape (n_examples, n_features) = (size, dims)
extword_size, word_dims = extword_embed.shape
logging.info("Load extword embed: words %d, dims %d." % (extword_size, word_dims))

测试，类 Vocab，函数 load_pretrained_embs 运行了


2020-07-26 12:47:46,161 INFO: Load extword embed: words 4337, dims 100.


In [99]:
# 额外的嵌入层，数值为训练过的 word2vec
extword_embed1 = nn.Embedding(extword_size, word_dims, padding_idx=0)
extword_embed1.weight.data.copy_(torch.from_numpy(extword_embed))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0446, -0.1373,  0.2730,  ...,  0.1510,  0.1139,  0.1337],
        [ 0.2177,  1.9787,  0.2219,  ...,  0.3089, -0.5231, -1.7075],
        ...,
        [ 2.4204,  0.6877,  0.4054,  ...,  0.2642,  0.4043, -1.1890],
        [-0.6415,  0.0572,  1.6200,  ...,  2.4325,  0.2091,  0.8547],
        [ 0.3198,  0.9130, -1.3433,  ...,  0.5710, -1.0892,  0.0587]])

不能直接对层进行索引，否则报错。但是前面的嵌入层为何可以？？

In [64]:
print(extword_embed1.weight.data.shape)
extword_embed1.weight.data[:3]

torch.Size([4337, 100])


tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

读入前面的vocab

In [29]:
import torch.nn as nn

#word2vec_path = './emb/word2vec.txt'
word2vec_path = './emb/word2vec_all.txt'
# 随机保留 or 舍弃 0.15 ？？？？
dropout = 0.15

class WordCNNEncoder(nn.Module):
    def __init__(self, vocab):
        """传入前面的 Vocab 类，在额外嵌入层时使用方法，加载训练好的 word2vec"""
        
        # 为什么都得继承一下？？？？？？？？？？？？？？？？？？
        super(WordCNNEncoder, self).__init__()
        
        self.dropout = nn.Dropout(dropout)
        self.word_dims = 100
        
        # word 对应的 Embedding 层的参数需要后面训练（forward），padding_idx（保持句子长度一致） 0填充？？？？？？？？
        # vocab.word_size 即不重复的单词的数目 4337 = 4335 + 2
        self.word_embed = nn.Embedding(vocab.word_size, self.word_dims, padding_idx=0)
        
        # 前面训练过的词向量（word2vec）作为额外的嵌入层，参数都是训练好的！！！！
        extword_embed = vocab.load_pretrained_embs(word2vec_path)
        extword_size, word_dims = extword_embed.shape   # (word_size, word_dims) = （4335+2， 100）
        logging.info("Load extword embed: words %d, dims %d." % (extword_size, word_dims))   
        self.extword_embed = nn.Embedding(extword_size, word_dims, padding_idx=0)
        self.extword_embed.weight.data.copy_(torch.from_numpy(extword_embed))
        # requires_grad ？？？？？？？？？？？？？？？
        self.extword_embed.weight.requires_grad = False
             
        # 不同的kernel可以获取不同范围内词的关系，获得的是纵向的差异信息，
        # 即类似于n-gram window，也就是在一个句子中不同范围的词出现会带来什么信息。比如可以使用3,4,5个词数分别作为卷积核的大小
        self.filter_sizes = [2, 3, 4]  
        self.out_channel = 100
        # 整个句子（嵌入层的一行）都被卷积
        input_size = self.word_dims
        # 3个卷积层的列表，对应不同尺寸的卷积核
        self.convs = nn.ModuleList([nn.Conv2d(1, self.out_channel, (filter_size, input_size), bias=True)
                                    for filter_size in self.filter_sizes])

    def forward(self, word_ids, extword_ids):
        """word_ids 即 model 类中的 batch_inputs， shape = (batch_size * max_doc_len, max_sent_len) =（sen_num，sent_len）"""
        # word_ids、extword_ids: （sen_num，sent_len） 
        
        # sent层？？？？？？？
        sen_num, sent_len = word_ids.shape
        
        # 向嵌入层内传入二维张量参数，变成三个维度 
        word_embed = self.word_embed(word_ids)        # （sen_num，sent_len，word_dims=100） 
        
        # 不是已经有网络参数了吗，怎么还要传入参数？？？？？？？？？？？
        extword_embed = self.extword_embed(extword_ids)
        # 名字对应 batch~！！！！！！！！！！！！！！！！！！！！
        batch_embed = word_embed + extword_embed
        
        # Dropout后，维度不变
        if self.training:
            batch_embed = self.dropout(batch_embed)     # （sen_num，sent_len，word_dims） 
        
        # unsqueeze 增加维度（相当于view），参数 1 是指增加到第二个维度，因为后面conv层要求的输入是4维张量
        batch_embed.unsqueeze_(1)  # （sen_num，1，sent_len，100）
        
        # 使用不同的卷积核，对应不同的卷积层
        pooled_outputs = []
        for i in range(len(self.filter_sizes)):
            # 卷积后得到的结果是一个vector，其 shape=(sentence_len - filter_window_size + 1, 1) 
            filter_height = sent_len - self.filter_sizes[i] + 1

            conv = self.convs[i](batch_embed)
            
            # 维度应该是不变的
            hidden = F.relu(conv)  # （sen_num，out_channel，filter_height，1）
            
            # 对应conv层是四维张量的输出，所以用二维池化（4维输入要求，一维池化是三维输入要求）  
            mp = nn.MaxPool2d((filter_height, 1))   # (filter_height, filter_width) = (filter_height, 1)
            
            # Max Pooling Over Time（1-max pooling）从卷积层一系列特征值中取最强的那个值
            # 四维张量化成二维张量，mp(hidden).shape = （sen_num，out_channel，1，1）
            pooled = mp(hidden).reshape(sen_num, self.out_channel)  # （sen_num，out_channel）
            pooled_outputs.append(pooled) 
        
        # 在第二个维度把这几个二维张量拼起来，reps 是？？？？？？？？？？？？？
        reps = torch.cat(pooled_outputs, dim=1)   # （sen_num，total_out_channel = out_channel * 3）

        if self.training:
            reps = self.dropout(reps)

        return reps

In [100]:
# 为什么keras实现那里用的是 MaxPool1d？？？？？？？？？
help(nn.MaxPool2d)

Help on class MaxPool2d in module torch.nn.modules.pooling:

class MaxPool2d(_MaxPoolNd)
 |  Applies a 2D max pooling over an input signal composed of several input
 |  planes.
 |  
 |  In the simplest case, the output value of the layer with input size :math:`(N, C, H, W)`,
 |  output :math:`(N, C, H_{out}, W_{out})` and :attr:`kernel_size` :math:`(kH, kW)`
 |  can be precisely described as:
 |  
 |  .. math::
 |      \begin{aligned}
 |          out(N_i, C_j, h, w) ={} & \max_{m=0, \ldots, kH-1} \max_{n=0, \ldots, kW-1} \\
 |                                  & \text{input}(N_i, C_j, \text{stride[0]} \times h + m,
 |                                                 \text{stride[1]} \times w + n)
 |      \end{aligned}
 |  
 |  If :attr:`padding` is non-zero, then the input is implicitly zero-padded on both sides
 |  for :attr:`padding` number of points. :attr:`dilation` controls the spacing between the kernel points.
 |  It is harder to describe, but this `link`_ has a nice visualization

#### build sent encoder

In [30]:
sent_hidden_size = 256
sent_num_layers = 2


class SentEncoder(nn.Module):
    def __init__(self, sent_rep_size):
        # ？？？？？？？？
        super(SentEncoder, self).__init__()
        
        self.dropout = nn.Dropout(dropout)
        
        # 加入lstm层，论文中也没有啊，lstm层的参数的含义？？？？？？？？？？？？？？
        self.sent_lstm = nn.LSTM(
            input_size=sent_rep_size,     # 输入的特征维度 300
            hidden_size=sent_hidden_size, # 隐藏层的特征维度 256
            num_layers=sent_num_layers,   # lstm隐藏层的层数 2
            batch_first=True,
            bidirectional=True
        )

    def forward(self, sent_reps, sent_masks):
        # sent_reps:  b x doc_len x sent_rep_size （batch_size, max_doc_len，sent_rep_size ) 可以reshape为（sent_num， sent_rep_size ）
        # sent_masks: b x doc_len
        
        # 返回的参数不止一个
        sent_hiddens, _ = self.sent_lstm(sent_reps)  # （b，doc_len，hidden*2）
        sent_hiddens = sent_hiddens * sent_masks.unsqueeze(2)

        if self.training:
            sent_hiddens = self.dropout(sent_hiddens)

        return sent_hiddens

#### build Attention

torch在参数初始化时，可以先给定shape，再给数值！！！！！！

In [7]:
help(torch.Tensor.masked_fill_)

Help on method_descriptor:

masked_fill_(...)
    masked_fill_(mask, value)
    
    Fills elements of :attr:`self` tensor with :attr:`value` where :attr:`mask` is
    True. The shape of :attr:`mask` must be
    :ref:`broadcastable <broadcasting-semantics>` with the shape of the underlying
    tensor.
    
    Args:
        mask (BoolTensor): the boolean mask
        value (float): the value to fill in with



In [5]:
(1 - torch.Tensor([1, 0])).bool()

tensor([False,  True])

类里的 nn.Module 是？？？？？？？？？？？？？？？？？？

是不是因为直接生成的tensor的参数**不能进行梯度训练**，所以先定义形状，再给数值？？ ？？？？？？？？

修改下行不？

https://www.jianshu.com/p/a105858567df

In [68]:
sent_hidden_size_ = 256
hidden_size_ = sent_hidden_size_ * 2
weight = torch.normal(mean=0, std=0.05, size=(hidden_size_, hidden_size_))
weight.requires_grad

False

In [66]:
bias = torch.zeros(hidden_size_)
bias.requires_grad

False

In [77]:
query = torch.normal(0.0, 0.05, (hidden_size_, )) 
query.requires_grad

False

In [70]:
weight.requires_grad = True
weight.requires_grad

True

In [31]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        # 超类,继承自己，没有从模块里啊 ？？？？？？？？？？？？？
        super(Attention, self).__init__()
        
        # hidden_size 为 Attention 层的隐藏单元数，为sent层中lstm层的隐藏单元数的两倍！！！！！！！！
        
        # 参数的初始化和 tf 不同，没有torch.normal，直接一步到位吗？？？？？？？
        self.weight = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.weight.data.normal_(mean=0.0, std=0.05)
        
        # 没有 torch.zeros ，为何不torch.Tensor(b) ？？？？？
        self.bias = nn.Parameter(torch.Tensor(hidden_size))
        b = np.zeros(hidden_size, dtype=np.float32)
        self.bias.data.copy_(torch.from_numpy(b))
        
        # query ？？？？
        self.query = nn.Parameter(torch.Tensor(hidden_size))
        self.query.data.normal_(mean=0.0, std=0.05)

    def forward(self, batch_hidden, batch_masks):
        # batch_hidden: (b，len，hidden_size=2 * hidden_size of lstm）
        # batch_masks:  （b，len）

        # linear
        key = torch.matmul(batch_hidden, self.weight) + self.bias  # （b，len，hidden） 

        # compute attention
        # query 是一维张量，所以key会减少一个维度
        outputs = torch.matmul(key, self.query)  # （b，len）
        
        # 将outputs中，对应True的位置，用float(-1e32)填充
        # https://blog.csdn.net/jianyingyao7658/article/details/103382654
        masked_outputs = outputs.masked_fill((1 - batch_masks).bool(), float(-1e32))  # （b，len）
        
        # 多分类，softmax 函数激活后再输出，dim=1 是对每一行进行 softmax 激活 ？？？？
        attn_scores = F.softmax(masked_outputs, dim=1)  # （b，len）

        # 对于全零向量，-1e32的结果为 1/len, -inf为nan, 额外补0 ????????????????????????????
        masked_attn_scores = attn_scores.masked_fill((1 - batch_masks).bool(), 0.0)   # （b，len）

        # sum weighted sources ？？？？？？？？？？
        # 批数据之间的矩阵乘法：（b，1, len） 与（b，len，hidden）的 torch.bmm 结果为 （b，1，hidden）
        batch_outputs = torch.bmm(masked_attn_scores.unsqueeze(1), key).squeeze(1)  # （b，1，hidden） 降维为（b，hidden）

        return batch_outputs, attn_scores

**torch.mm  torch.matmul  torch.bmm 之间的区别**

https://blog.csdn.net/ganxiwu9686/article/details/95204013

In [ ]:
torch.view()即tf.reshape

#### build model

In [36]:
word_encoder = WordCNNEncoder(vocab)

测试，类 Vocab，函数 word_size 运行了
测试，类 Vocab，函数 load_pretrained_embs 运行了


2020-07-29 10:48:48,719 INFO: Load extword embed: words 5978, dims 100.


5978 5978


In [37]:
print(word_encoder)

WordCNNEncoder(
  (dropout): Dropout(p=0.15, inplace=False)
  (word_embed): Embedding(5996, 100, padding_idx=0)
  (extword_embed): Embedding(5978, 100, padding_idx=0)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  )
)


In [12]:
print(word_encoder)

WordCNNEncoder(
  (dropout): Dropout(p=0.15, inplace=False)
  (word_embed): Embedding(4337, 100, padding_idx=0)
  (extword_embed): Embedding(4337, 100, padding_idx=0)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  )
)


In [39]:
word_encoder.state_dict()

OrderedDict([('word_embed.weight',
              tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
                      [-0.1352, -0.3804, -0.0035,  ..., -1.3826,  0.5351,  1.0329],
                      [ 0.9951, -0.3827,  2.6233,  ..., -1.0207,  1.9715,  0.1106],
                      ...,
                      [-0.5620,  0.3514, -0.4366,  ...,  1.6153, -0.0592,  0.2314],
                      [ 1.2819,  2.1936,  0.3616,  ...,  0.0555,  0.8765, -0.8002],
                      [-1.5785, -0.2578, -1.2632,  ...,  1.9032,  0.1489, -1.2119]])),
             ('extword_embed.weight',
              tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
                      [-0.0446, -0.1373,  0.2730,  ...,  0.1510,  0.1139,  0.1337],
                      [ 0.2177,  1.9787,  0.2219,  ...,  0.3089, -0.5231, -1.7075],
                      ...,
                      [ 2.4204,  0.6877,  0.4054,  ...,  0.2642,  0.4043, -1.1890],
                      [-0.6415

In [38]:
# <generator object Module.parameters at 0x000001D415D19E08>
len(list(word_encoder.parameters())), list(word_encoder.parameters())

(8, [Parameter containing:
  tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.1352, -0.3804, -0.0035,  ..., -1.3826,  0.5351,  1.0329],
          [ 0.9951, -0.3827,  2.6233,  ..., -1.0207,  1.9715,  0.1106],
          ...,
          [-0.5620,  0.3514, -0.4366,  ...,  1.6153, -0.0592,  0.2314],
          [ 1.2819,  2.1936,  0.3616,  ...,  0.0555,  0.8765, -0.8002],
          [-1.5785, -0.2578, -1.2632,  ...,  1.9032,  0.1489, -1.2119]],
         requires_grad=True), Parameter containing:
  tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.0446, -0.1373,  0.2730,  ...,  0.1510,  0.1139,  0.1337],
          [ 0.2177,  1.9787,  0.2219,  ...,  0.3089, -0.5231, -1.7075],
          ...,
          [ 2.4204,  0.6877,  0.4054,  ...,  0.2642,  0.4043, -1.1890],
          [-0.6415,  0.0572,  1.6200,  ...,  2.4325,  0.2091,  0.8547],
          [ 0.3198,  0.9130, -1.3433,  ...,  0.5710, -1.0892,  0.0587]]), Parameter containing:
  

In [42]:
a = list(filter(lambda p: p.requires_grad, word_encoder.parameters()))
print(len(a))
a[0]

7


Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1352, -0.3804, -0.0035,  ..., -1.3826,  0.5351,  1.0329],
        [ 0.9951, -0.3827,  2.6233,  ..., -1.0207,  1.9715,  0.1106],
        ...,
        [-0.5620,  0.3514, -0.4366,  ...,  1.6153, -0.0592,  0.2314],
        [ 1.2819,  2.1936,  0.3616,  ...,  0.0555,  0.8765, -0.8002],
        [-1.5785, -0.2578, -1.2632,  ...,  1.9032,  0.1489, -1.2119]],
       requires_grad=True)

**word_encoder层的情况：**
- Dropout 层没有参数
- 除了extword_embed 层不需要训练外，其他的都需要

In [38]:
print(len(list(word_encoder.parameters())))
for parameter_name, parameter in zip(word_encoder.state_dict().keys(), word_encoder.parameters()):
    print(parameter_name)
    print(parameter.size(), '\t', parameter.requires_grad)
    # 比 \n 的空行短
    print()

8
word_embed.weight
torch.Size([5996, 100]) 	 True

extword_embed.weight
torch.Size([5978, 100]) 	 False

convs.0.weight
torch.Size([100, 1, 2, 100]) 	 True

convs.0.bias
torch.Size([100]) 	 True

convs.1.weight
torch.Size([100, 1, 3, 100]) 	 True

convs.1.bias
torch.Size([100]) 	 True

convs.2.weight
torch.Size([100, 1, 4, 100]) 	 True

convs.2.bias
torch.Size([100]) 	 True



**sent层的参数情况：**
- 两个隐藏层10和11，都需要训练

In [52]:
sent_rep_size = 300
sent = SentEncoder(sent_rep_size)
# TypeError: object of type 'generator' has no len()
print(len(list(sent.parameters())))
for parameter_name, parameter in zip(sent.state_dict().keys(), sent.parameters()):
    print(parameter_name)
    print(parameter.size(), '\t', parameter.requires_grad)
    print()

16
sent_lstm.weight_ih_l0
torch.Size([1024, 300]) 	 True

sent_lstm.weight_hh_l0
torch.Size([1024, 256]) 	 True

sent_lstm.bias_ih_l0
torch.Size([1024]) 	 True

sent_lstm.bias_hh_l0
torch.Size([1024]) 	 True

sent_lstm.weight_ih_l0_reverse
torch.Size([1024, 300]) 	 True

sent_lstm.weight_hh_l0_reverse
torch.Size([1024, 256]) 	 True

sent_lstm.bias_ih_l0_reverse
torch.Size([1024]) 	 True

sent_lstm.bias_hh_l0_reverse
torch.Size([1024]) 	 True

sent_lstm.weight_ih_l1
torch.Size([1024, 512]) 	 True

sent_lstm.weight_hh_l1
torch.Size([1024, 256]) 	 True

sent_lstm.bias_ih_l1
torch.Size([1024]) 	 True

sent_lstm.bias_hh_l1
torch.Size([1024]) 	 True

sent_lstm.weight_ih_l1_reverse
torch.Size([1024, 512]) 	 True

sent_lstm.weight_hh_l1_reverse
torch.Size([1024, 256]) 	 True

sent_lstm.bias_ih_l1_reverse
torch.Size([1024]) 	 True

sent_lstm.bias_hh_l1_reverse
torch.Size([1024]) 	 True



**Attention层的参数情况：**
- 都需要训练

In [57]:
# 256 * 2
doc_rep_size = sent_hidden_size * 2
attention = Attention(doc_rep_size)
print(len(list(attention.parameters())))
for parameter_name, parameter in zip(attention.state_dict().keys(), attention.parameters()):
    print(parameter_name)
    print(parameter.size(), '\t', parameter.requires_grad)
    print()

3
weight
torch.Size([512, 512]) 	 True

bias
torch.Size([512]) 	 True

query
torch.Size([512]) 	 True



In [80]:
[np.prod(list(parameter.size())) for parameter in attention.parameters()]

[262144, 512, 512]

In [32]:
class Model(nn.Module):
    def __init__(self, vocab):
        super(Model, self).__init__()
        
        # # 输入的特征维度 300=100*3，3个卷积后的特征拼接
        self.sent_rep_size = 300
        # hidden_size 为 Attention 层的隐藏单元数，为sent层中lstm层的隐藏单元数的两倍
        self.doc_rep_size = sent_hidden_size * 2
        
        self.all_parameters = {}
        
        # 把 requires_grad 的层的参数都存到这里
        parameters = [] 
        # 2 2
        # print(len(set(vocab._id2extword)), len(vocab._id2extword))
        self.word_encoder = WordCNNEncoder(vocab)       
        parameters.extend(list(filter(lambda p: p.requires_grad, self.word_encoder.parameters())))
        self.sent_encoder = SentEncoder(self.sent_rep_size)        
        parameters.extend(list(filter(lambda p: p.requires_grad, self.sent_encoder.parameters())))        
        self.sent_attention = Attention(self.doc_rep_size)
        parameters.extend(list(filter(lambda p: p.requires_grad, self.sent_attention.parameters())))
        # 输出层使用全连接层，14 分类
        self.out = nn.Linear(self.doc_rep_size, vocab.label_size, bias=True)
        parameters.extend(list(filter(lambda p: p.requires_grad, self.out.parameters())))

        if use_cuda:
            self.to(device)
        
        # 全部待训练的参数
        if len(parameters) > 0:
            self.all_parameters["basic_parameters"] = parameters

        logging.info('Build model with cnn word encoder, lstm sent encoder.')
    
        # 这里的 self 是 model 实例，直接调用未定义的方法？？？？？？？？？？？？？？？
        para_num = sum([np.prod(list(p.size())) for p in self.parameters()])
        print([np.prod(list(p.size())) for p in self.parameters()])
        
        # 从多少个字节 b，转换到多少 Mb
        logging.info('Model param num: %.2f M.' % (para_num / 1e6))
        logging.info('Model param num: %.2f M.' % (para_num / 1024 / 1024))

    def forward(self, batch_inputs):
        """doc_len = word_dims？？？？？？？"""
        # batch_inputs 包含三个 shape 一样的变量（batch_inputs1, batch_inputs2, batch_masks）
        
        # batch_inputs(batch_inputs1, batch_inputs2): b x doc_len x sent_len
        # batch_masks : b x doc_len x sent_len
        
        batch_inputs1, batch_inputs2, batch_masks = batch_inputs
        
        # b = batch_size, doc_len = max_doc_len, sent_len = max_sent_len
        # 同 batch_inputs2, batch_masks， batch_inputs1.shape = (b，doc_len，sent_len)  
        batch_size, max_doc_len, max_sent_len = batch_inputs1.shape[0], batch_inputs1.shape[1], batch_inputs1.shape[2]
        
        # reshape：sent_num = batch_size * max_doc_len
        batch_inputs1 = batch_inputs1.view(batch_size * max_doc_len, max_sent_len)  # （sen_num，sent_len）
        batch_inputs2 = batch_inputs2.view(batch_size * max_doc_len, max_sent_len)  # （sen_num，sent_len）
        batch_masks = batch_masks.view(batch_size * max_doc_len, max_sent_len)  # （sen_num，sent_len）
        
        # word_encoder层传入参数，函数式编程
        sent_reps = self.word_encoder(batch_inputs1, batch_inputs2)  # （sen_num，sent_rep_size=100*3）

        sent_reps = sent_reps.view(batch_size, max_doc_len, self.sent_rep_size)  # b x doc_len x sent_rep_size
        batch_masks = batch_masks.view(batch_size, max_doc_len, max_sent_len)  # b x doc_len x max_sent_len
        sent_masks = batch_masks.bool().any(2).float()  # b x doc_len

        sent_hiddens = self.sent_encoder(sent_reps, sent_masks)  # b x doc_len x doc_rep_size
        doc_reps, atten_scores = self.sent_attention(sent_hiddens, sent_masks)  # b x doc_rep_size

        batch_outputs = self.out(doc_reps)  # b x num_labels

        return batch_outputs

In [33]:
# self就是总的模型， self.parameters() 的个数 = 29 = 8 + 16 + 3 + 2
model = Model(vocab)

测试，类 Vocab，函数 word_size 运行了
测试，类 Vocab，函数 load_pretrained_embs 运行了


2020-07-29 10:45:49,424 INFO: Load extword embed: words 5978, dims 100.
2020-07-29 10:45:49,474 INFO: Build model with cnn word encoder, lstm sent encoder.
2020-07-29 10:45:49,476 INFO: Model param num: 4.28 M.
2020-07-29 10:45:49,476 INFO: Model param num: 4.08 M.


5978 5978
测试，类 Vocab，函数 label_size 运行了
[599600, 597800, 20000, 100, 30000, 100, 40000, 100, 307200, 262144, 1024, 1024, 307200, 262144, 1024, 1024, 524288, 262144, 1024, 1024, 524288, 262144, 1024, 1024, 262144, 512, 512, 7168, 14]


In [ ]:
word_encoder每运行一次，都会增加，导致结果不一致 assert len(set(self._id2extword)) == len(self._id2extword)

需要重新运行一次vocab才行